# 第三题：实现带有拉普拉斯修正的朴素贝叶斯

实验内容：
1. 叙述拉普拉斯修正的作用
2. 给出使用的数据集
3. 给出实现的代码，要有详细的注释
4. 给出模型评价指标的结果

In [1]:
# YOUR CODE HERE
import numpy as np
spambase = np.loadtxt('data/spambase/spambase.data', delimiter = ",")
spamx = spambase[:, :57]
spamy = spambase[:, 57]
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(spamx, spamy, test_size = 0.4, random_state = 32)

In [2]:
# YOUR CODE HERE
class myGaussianNB:
    '''
    处理连续特征的带拉普拉斯修正的高斯朴素贝叶斯
    '''
    def __init__(self, alpha=1.0):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.mean              类标记 与 均值(np.ndarray)
        self.var               类标记 与 方差(np.ndarray)
        
        Parameters
        ----------
        alpha: float, 拉普拉斯平滑的参数
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.mean = dict()
        self.var = dict()
        self.alpha = alpha  # 拉普拉斯平滑的参数
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.mean.clear()
        self.var.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率，以及这类训练样本每个特征的均值和方差

        Parameters
        ----------
            trainX: np.ndarray, 训练样本的特征, 维度：(样本数, 特征数)
        
            trainY: np.ndarray, 训练样本的标记, 维度：(样本数, )
        '''
        
        # 先调用_clear
        self._clear()
        
        # 获取类标记
        labels = np.unique(trainY)
        
        # 添加类标记与下标的映射关系
        self.label_mapping = {label: index for index, label in enumerate(labels)}
        
        # 遍历每个类
        for label in labels:
            
            # 取出为label这类的所有训练样本，存为 x
            x = trainX[trainY == label, :]
            
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率
            self.probability_of_y[label] = (x.shape[0] + self.alpha) / (trainX.shape[0] + len(labels) * self.alpha)
            
            # 对 x 的每列求均值，使用 keepdims = True 保持维度，存储到 self.mean 中，键为 label，值为每列的均值组成的一个二维 np.ndarray
            self.mean[label] = np.mean(x, axis=0, keepdims=True)
            
            # 这句话是debug用的，如果不满足下面的条件，会直接跳出
            assert self.mean[label].shape == (1, trainX.shape[1])
            
            # 对 x 的每列求方差，使用 keepdims = True 保持维度，存储到 self.var 中，键为 label，值为每列的方差组成的一个二维 np.ndarray
            self.var[label] = np.var(x, axis=0, keepdims=True)
            
            # debug
            assert self.var[label].shape == (1, trainX.shape[1])
            
            # 平滑，因为方差在公式的分母部分，我们要加一个很小的数，防止除以0
            self.var[label] += self.alpha
        
    def predict(self, testX):
        '''
        给定测试样本，预测测试样本的类标记，这里我们要实现化简后的公式

        Parameters
        ----------
            testX: np.ndarray, 测试的特征, 维度：(测试样本数, 特征数)
    
        Returns
        ----------
            prediction: np.ndarray, 预测结果, 维度：(测试样本数, )
        '''
        
        # 初始化一个空矩阵 results，存储每个样本属于每个类的概率，维度是 (测试样本数，类别数)，每行表示一个样本，每列表示一个特征
        results = np.empty((testX.shape[0], len(self.probability_of_y)))
        
        # 初始化一个列表 labels，按 self.label_mapping 的映射关系存储所有的标记，一会儿会在下面的循环内部完成存储
        labels = [0] * len(self.probability_of_y)
        
        # 遍历当前的类，label为类标记，index为下标，我们将每个样本预测出来的这个 label 的概率，存到 results 中的第 index 列
        for label, index in self.label_mapping.items():
            
            # 先验概率存为 py
            py = self.probability_of_y[label]
            
            # 使用变换后的公式，计算所有特征的条件概率之和，存为sum_of_conditional_probability
            sum_of_conditional_probability = np.sum(-0.5 * np.log(2 * np.pi * self.var[label]) - 0.5 * ((testX - self.mean[label]) / self.var[label]) ** 2, axis=1)
            
            # 使用变换后的公式，将 条件概率 与 log先验概率 相加，存为result，维度应该是 (测试样本数, )
            result = sum_of_conditional_probability + np.log(py)
            
            # 将所有测试样本属于当前这类的概率，存入到results中
            results[:, index] = result
            
            # 将当前的label，按index顺序放入到labels中
            labels[index] = label
        
        # 将labels转换为np.ndarray
        np_labels = np.array(labels)
        
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中
        max_prob_index = np.argmax(results, axis=1)
        
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        prediction = np_labels[max_prob_index]
        
        # 返回预测结果
        return prediction


In [3]:
# YOUR CODE HERE
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 假设 model 是通过 myGaussianNB 训练好的模型
model = myGaussianNB()
model.fit(trainX, trainY)

# 假设 testX 和 testY 是测试集
predicted_labels = model.predict(testX)

# 计算准确率
accuracy = accuracy_score(testY, predicted_labels)
# 计算查准率
precision = precision_score(testY, predicted_labels, average='binary')  
# 计算查全率
recall = recall_score(testY, predicted_labels, average='binary')  
# 计算 F1 值
f1 = f1_score(testY, predicted_labels, average='binary')  

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.7246
Precision: 0.8868
Recall: 0.3301
F1 Score: 0.4811
